# Tutorial: Creating a Hybrid Retrieval Pipeline

- **Level**: Intermediate
- **Time to complete**: 15 minutes
- **Components Used**: [`DocumentSplitter`](https://docs.haystack.deepset.ai/docs/documentsplitter), [`SentenceTransformersDocumentEmbedder`](https://docs.haystack.deepset.ai/docs/sentencetransformersdocumentembedder), [`DocumentJoiner`](https://docs.haystack.deepset.ai/docs/documentjoiner), [`InMemoryDocumentStore`](https://docs.haystack.deepset.ai/docs/inmemorydocumentstore), [`InMemoryBM25Retriever`](https://docs.haystack.deepset.ai/docs/inmemorybm25retriever), [`InMemoryEmbeddingRetriever`](https://docs.haystack.deepset.ai/docs/inmemoryembeddingretriever), and [`TransformersSimilarityRanker`](https://docs.haystack.deepset.ai/docs/transformerssimilarityranker)
- **Prerequisites**: None
- **Goal**: After completing this tutorial, you will have learned about creating a hybrid retrieval and when it's useful.

In [1]:
import os
import sys
import warnings

import pandas as pd

from tqdm import tqdm
from uuid import uuid4

from langchain_chroma import Chroma
# from langchain_huggingface import HuggingFaceEmbeddings

# from langchain_core.documents import Document

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

# import log_files
from data_processing import DataProcessing
from text_generation_models import TextGenerationModelFactory

In [2]:
pd.set_option('max_colwidth', 800)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [3]:
prediction_properties = DataProcessing.load_prediction_properties()
prediction_properties

' A prediction <p> = (<p_s>, <p_t>, <p_d>, <p_o>), where it consists of the following four properties:\n\n        1. <p_s>\n            - Defined as: \n                - Source entity that states the <p>\n            - Characteristics:\n                - A person with either: a name only, profile name only, geneder only, domain specific title only or any combination of these.\n                - An associated organization\n                - Named entity: Person, organization\n                - Part of speech: Noun\n            - Examples:\n                1. A person with a name only: Detravious\n                2. A person with a profile name: FitToJesus\n                3. A person with a gender only: He\n                4. A person with a domain specific title: reporter, analyst, expert, top executive, senior level person, etc \n                5. A person with a combination: Detravious, a reporter\n                6. An associated organization: FitTo...\n                7. A combina

In [4]:
# path = os.path.join(notebook_dir, "../data/rag/retrieved/", "retrieved_results-v1.csv")
# # prefix = "text_label_metadata"
# save_file_type = "csv"
# df = DataProcessing.load_from_file(path, save_file_type)
# df.drop(['Unnamed: 0'], axis=1, inplace=True)


path = os.path.join(notebook_dir, "../data/rag/retrieved/", "text_label_name_meta_data-v1.csv")
# prefix = "text_label_metadata"
save_file_type = "csv"
df = DataProcessing.load_from_file(path, save_file_type)
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df

,text,llm_label,llm_name,meta_data
0,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is aggressively pursuing its growth strategy by increasingly focusing on technologically more demanding HDI printed circuit boards PCBs .,prediction,llama-3.1-8b-instant,"id 084eec7989f0cfbbb94860b54ffffe5c0ea064f896a8dc4e4e69dfa1f4fb6eb0\nblob NaN\nmeta {'sentiment': 'positive', 'source_id': '2c106b768ca3455523070e94cfcdbc9e145ca808ae79f54e5d2f7c56e21ab280', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0, '_split_overlap': []}\nscore ..."
1,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is aggressively pursuing its growth strategy by increasingly focusing on technologically more demanding HDI printed circuit boards PCBs .,Non-prediction,llama-3.3-70b-versatile,"id 084eec7989f0cfbbb94860b54ffffe5c0ea064f896a8dc4e4e69dfa1f4fb6eb0\nblob NaN\nmeta {'sentiment': 'positive', 'source_id': '2c106b768ca3455523070e94cfcdbc9e145ca808ae79f54e5d2f7c56e21ab280', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0, '_split_overlap': []}\nscore ..."
2,"Net sales surged by 18.5 % to EUR167 .8 m. Teleste said that EUR20 .4 m , or 12.2 % , of the sales came from the acquisitions made in 2009 .",prediction,llama-3.1-8b-instant,id 53f4afca60c6d97e69fe7bf80b319601dfaed593c...
3,"Net sales surged by 18.5 % to EUR167 .8 m. Teleste said that EUR20 .4 m , or 12.2 % , of the sales came from the acquisitions made in 2009 .",Non-prediction,llama-3.3-70b-versatile,id 53f4afca60c6d97e69fe7bf80b319601dfaed593c...
4,"According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .",prediction,llama-3.1-8b-instant,id c8250eb1bfc021b035d71ba172e262a21ab427b75...
5,"According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .",Prediction,llama-3.3-70b-versatile,id c8250eb1bfc021b035d71ba172e262a21ab427b75...
6,"The international electronic industry company Elcoteq has laid off tens of employees from its Tallinn facility ; contrary to earlier layoffs the company contracted the ranks of its office workers , the daily Postimees reported .",prediction,llama-3.1-8b-instant,id bc1d4924ecbc2356699cdec2c4662ea3f2125a2dc...
7,"The international electronic industry company Elcoteq has laid off tens of employees from its Tallinn facility ; contrary to earlier layoffs the company contracted the ranks of its office workers , the daily Postimees reported .",Non-prediction,llama-3.3-70b-versatile,id bc1d4924ecbc2356699cdec2c4662ea3f2125a2dc...
8,"Finnish Talentum reports its operating profit increased to EUR 20.5 mn in 2005 from EUR 9.3 mn in 2004 , and net sales totaled EUR 103.3 mn , up from EUR 96.4 mn .",prediction,llama-3.1-8b-instant,"id cda0ab041b78439da23a9559adf5a33bc578eb7080be3bf050270d043d491cce\nblob NaN\nmeta {'sentiment': 'positive', 'source_id': 'c4fff1f5fb136e3170ef2af33b26fe54a73fcf7f1f8feb8458dcf57c6fd3d7f8', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0, '_split_overlap': []}\nscore ..."
9,"Finnish Talentum reports its operating profit increased to EUR 20.5 mn in 2005 from EUR 9.3 mn in 2004 , and net sales totaled EUR 103.3 mn , up from EUR 96.4 mn .",Non-prediction,llama-3.3-70b-versatile,"id cda0ab041b78439da23a9559adf5a33bc578eb7080be3bf050270d043d491cce\nblob NaN\nmeta {'sentiment': 'positive', 'source_id': 'c4fff1f5fb136e3170ef2af33b26fe54a73fcf7f1f8feb8458dcf57c6fd3d7f8', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0, '_split_overlap': []}\nscore ..."


## Filter for No Labels

In [5]:
na_filt = df['llm_label'].isna()
na_filt
no_labels_df = df[na_filt]
no_labels_df

,text,llm_label,llm_name,meta_data
40,"In the third quarter of 2010 , net sales increased by 5.2 % to EUR 205.5 mn , and operating profit by 34.9 % to EUR 23.5 mn .",NaN,NaN,id e686de84900a7b27a7d098eabd3963f222d02b200...
41,"Its board of directors will propose a dividend of EUR0 .12 per share for 2010 , up from the EUR0 .08 per share paid in 2009 .",NaN,NaN,id 00098507dbc5241dc1f90f8635bcbfb77307f21c7...
42,Net sales increased to EUR193 .3 m from EUR179 .9 m and pretax profit rose by 34.2 % to EUR43 .1 m. ( EUR1 = USD1 .4 ),NaN,NaN,id 3ddf930c3b3e084b53333715ff8a9f098e1bce74b...
43,With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .,NaN,NaN,id 9e8da179f1923e36a0092be7b7b424377e1daae08...
44,"Incap Contract Manufacturing Services Pvt Ltd , a subsidiary of Incap Corporation of Finland , plans to double its revenues by 2007-2008 .",NaN,NaN,id 9fc8c8f0070f5991c474874ace926849724c41842...
45,"Net income from life insurance doubled to EUR 6.8 mn from EUR 3.2 mn , and net income from non-life insurance rose to EUR 5.2 mn from EUR 1.5 mn in the corresponding period in 2009 .",NaN,NaN,id c9ed0b12181b55b02d6fe4c14be091b69ec548430...
46,"STORA ENSO , NORSKE SKOG , M-REAL , UPM-KYMMENE Credit Suisse First Boston ( CFSB ) raised the fair value for shares in four of the largest Nordic forestry groups .",NaN,NaN,id b0426be8586b1363cc40a132d93ecb6fa082d7403...
47,"Consolidated net sales increased 16 % to reach EUR74 .8 m , while operating profit amounted to EUR0 .9 m compared to a loss of EUR0 .7 m in the prior year period .",NaN,NaN,id 56b3589d22f351cc29abbff1be37d7353f5a20a10...
48,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is aggressively pursuing its growth strategy by increasingly focusing on technologically more demanding HDI printed circuit boards PCBs .,NaN,NaN,id 084eec7989f0cfbbb94860b54ffffe5c0ea064f89...
49,"According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .",NaN,NaN,id c8250eb1bfc021b035d71ba172e262a21ab427b75...


In [6]:
# no_labels_df['meta']

## LLM Certification

In [7]:
tgmf = TextGenerationModelFactory()



# Groq Cloud (https://console.groq.com/docs/overview)
llama_318b_instant_generation_model = tgmf.create_instance('llama-3.1-8b-instant') 
llama_3370b_versatile_generation_model = tgmf.create_instance('llama-3.3-70b-versatile')  
# llama_guard_4_12b_generation_model = tgmf.create_instance('meta-llama/llama-guard-4-12b')  

# models = [gemma_29b_generation_model, llama_318b_instant_generation_model, llama_3370b_versatile_generation_model, llama_guard_4_12b_generation_model]

# models = [gemma_29b_generation_model, llama_318b_instant_generation_model, llama_3370b_versatile_generation_model]
models = [llama_318b_instant_generation_model, llama_3370b_versatile_generation_model]

# NaviGator
# llama_31_70b_instruct = tgmf.create_instance('llama-3.1-70b-instruct') 
# mistral_small_31 = tgmf.create_instance('mistral-small-3.1') 
# models = [llama_31_70b_instruct, mistral_small_31]

In [8]:
def llm_certifier(data: str, prediction_properties: str, model):
    
        prompt = f"""We have {prediction_properties} \nGiven this and the sentence '{data}', classify if the {data} is a prediction or non-prediction.
        Do NOT generate anything more than these to classes (prediction or non-prediction). Do not state a reason for your classification of the text. Do not regenerate the text either."""
        # print(f"\tPrompt: {prompt}")
        # print()

        input_prompt = model.user(prompt)
        # print(input_prompt)  

        raw_text_llm_generation = model.chat_completion([input_prompt])
        # print(raw_text_llm_generation)

        for line in raw_text_llm_generation.split("\n"):
            return line

In [9]:
results = []
for idx, row in no_labels_df.iterrows():
    text = row['text']
    # meta_data = row.drop(labels=['text', 'llm_label', 'llm_name'])
    meta_data = row['meta_data']
    print(idx, text)

    for model in models:
        # print(idx, result)
        llm_label = llm_certifier(text, prediction_properties, model)
        print('\t', model.__name__(), '---', llm_label)
        result = (text, llm_label, model.__name__(), meta_data)
        # result = (text, llm_label, model.__name__())
        results.append(result)

40 In the third quarter of 2010 , net sales increased by 5.2 % to EUR 205.5 mn , and operating profit by 34.9 % to EUR 23.5 mn .
	 llama-3.1-8b-instant --- Prediction
	 llama-3.3-70b-versatile --- Non-prediction
41 Its board of directors will propose a dividend of EUR0 .12 per share for 2010 , up from the EUR0 .08 per share paid in 2009 .
	 llama-3.1-8b-instant --- prediction
	 llama-3.3-70b-versatile --- Prediction
42 Net sales increased to EUR193 .3 m from EUR179 .9 m and pretax profit rose by 34.2 % to EUR43 .1 m. ( EUR1 = USD1 .4 )
	 llama-3.1-8b-instant --- prediction
	 llama-3.3-70b-versatile --- Non-prediction
43 With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .
	 llama-3.1-8b-instant --- prediction
	 llama-3.3-70b-versatile --- Prediction
44 Incap Contract Manufacturing Services Pvt Ltd , a subsidiary of Incap Corporation o

In [10]:
# results_with_llm_label_df = pd.DataFrame(results, columns=['text', 'llm_label', 'llm_name', 'meta_data'])
# results_with_llm_label_df
new_results_df = pd.DataFrame(results, columns=['text', 'llm_label', 'llm_name', 'meta_data'])
new_results_df

,text,llm_label,llm_name,meta_data
0,"In the third quarter of 2010 , net sales increased by 5.2 % to EUR 205.5 mn , and operating profit by 34.9 % to EUR 23.5 mn .",Prediction,llama-3.1-8b-instant,id e686de84900a7b27a7d098eabd3963f222d02b200...
1,"In the third quarter of 2010 , net sales increased by 5.2 % to EUR 205.5 mn , and operating profit by 34.9 % to EUR 23.5 mn .",Non-prediction,llama-3.3-70b-versatile,id e686de84900a7b27a7d098eabd3963f222d02b200...
2,"Its board of directors will propose a dividend of EUR0 .12 per share for 2010 , up from the EUR0 .08 per share paid in 2009 .",prediction,llama-3.1-8b-instant,id 00098507dbc5241dc1f90f8635bcbfb77307f21c7...
3,"Its board of directors will propose a dividend of EUR0 .12 per share for 2010 , up from the EUR0 .08 per share paid in 2009 .",Prediction,llama-3.3-70b-versatile,id 00098507dbc5241dc1f90f8635bcbfb77307f21c7...
4,Net sales increased to EUR193 .3 m from EUR179 .9 m and pretax profit rose by 34.2 % to EUR43 .1 m. ( EUR1 = USD1 .4 ),prediction,llama-3.1-8b-instant,id 3ddf930c3b3e084b53333715ff8a9f098e1bce74b...
5,Net sales increased to EUR193 .3 m from EUR179 .9 m and pretax profit rose by 34.2 % to EUR43 .1 m. ( EUR1 = USD1 .4 ),Non-prediction,llama-3.3-70b-versatile,id 3ddf930c3b3e084b53333715ff8a9f098e1bce74b...
6,With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .,prediction,llama-3.1-8b-instant,id 9e8da179f1923e36a0092be7b7b424377e1daae08...
7,With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .,Prediction,llama-3.3-70b-versatile,id 9e8da179f1923e36a0092be7b7b424377e1daae08...
8,"Incap Contract Manufacturing Services Pvt Ltd , a subsidiary of Incap Corporation of Finland , plans to double its revenues by 2007-2008 .",Prediction,llama-3.1-8b-instant,id 9fc8c8f0070f5991c474874ace926849724c41842...
9,"Incap Contract Manufacturing Services Pvt Ltd , a subsidiary of Incap Corporation of Finland , plans to double its revenues by 2007-2008 .",Prediction,llama-3.3-70b-versatile,id 9fc8c8f0070f5991c474874ace926849724c41842...


In [11]:
llm_labelled_df = df.dropna()
llm_labelled_df

,text,llm_label,llm_name,meta_data
0,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is aggressively pursuing its growth strategy by increasingly focusing on technologically more demanding HDI printed circuit boards PCBs .,prediction,llama-3.1-8b-instant,"id 084eec7989f0cfbbb94860b54ffffe5c0ea064f896a8dc4e4e69dfa1f4fb6eb0\nblob NaN\nmeta {'sentiment': 'positive', 'source_id': '2c106b768ca3455523070e94cfcdbc9e145ca808ae79f54e5d2f7c56e21ab280', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0, '_split_overlap': []}\nscore ..."
1,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is aggressively pursuing its growth strategy by increasingly focusing on technologically more demanding HDI printed circuit boards PCBs .,Non-prediction,llama-3.3-70b-versatile,"id 084eec7989f0cfbbb94860b54ffffe5c0ea064f896a8dc4e4e69dfa1f4fb6eb0\nblob NaN\nmeta {'sentiment': 'positive', 'source_id': '2c106b768ca3455523070e94cfcdbc9e145ca808ae79f54e5d2f7c56e21ab280', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0, '_split_overlap': []}\nscore ..."
2,"Net sales surged by 18.5 % to EUR167 .8 m. Teleste said that EUR20 .4 m , or 12.2 % , of the sales came from the acquisitions made in 2009 .",prediction,llama-3.1-8b-instant,id 53f4afca60c6d97e69fe7bf80b319601dfaed593c...
3,"Net sales surged by 18.5 % to EUR167 .8 m. Teleste said that EUR20 .4 m , or 12.2 % , of the sales came from the acquisitions made in 2009 .",Non-prediction,llama-3.3-70b-versatile,id 53f4afca60c6d97e69fe7bf80b319601dfaed593c...
4,"According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .",prediction,llama-3.1-8b-instant,id c8250eb1bfc021b035d71ba172e262a21ab427b75...
5,"According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .",Prediction,llama-3.3-70b-versatile,id c8250eb1bfc021b035d71ba172e262a21ab427b75...
6,"The international electronic industry company Elcoteq has laid off tens of employees from its Tallinn facility ; contrary to earlier layoffs the company contracted the ranks of its office workers , the daily Postimees reported .",prediction,llama-3.1-8b-instant,id bc1d4924ecbc2356699cdec2c4662ea3f2125a2dc...
7,"The international electronic industry company Elcoteq has laid off tens of employees from its Tallinn facility ; contrary to earlier layoffs the company contracted the ranks of its office workers , the daily Postimees reported .",Non-prediction,llama-3.3-70b-versatile,id bc1d4924ecbc2356699cdec2c4662ea3f2125a2dc...
8,"Finnish Talentum reports its operating profit increased to EUR 20.5 mn in 2005 from EUR 9.3 mn in 2004 , and net sales totaled EUR 103.3 mn , up from EUR 96.4 mn .",prediction,llama-3.1-8b-instant,"id cda0ab041b78439da23a9559adf5a33bc578eb7080be3bf050270d043d491cce\nblob NaN\nmeta {'sentiment': 'positive', 'source_id': 'c4fff1f5fb136e3170ef2af33b26fe54a73fcf7f1f8feb8458dcf57c6fd3d7f8', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0, '_split_overlap': []}\nscore ..."
9,"Finnish Talentum reports its operating profit increased to EUR 20.5 mn in 2005 from EUR 9.3 mn in 2004 , and net sales totaled EUR 103.3 mn , up from EUR 96.4 mn .",Non-prediction,llama-3.3-70b-versatile,"id cda0ab041b78439da23a9559adf5a33bc578eb7080be3bf050270d043d491cce\nblob NaN\nmeta {'sentiment': 'positive', 'source_id': 'c4fff1f5fb136e3170ef2af33b26fe54a73fcf7f1f8feb8458dcf57c6fd3d7f8', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0, '_split_overlap': []}\nscore ..."


In [12]:
updated_df = pd.concat([llm_labelled_df, new_results_df])
updated_df

,text,llm_label,llm_name,meta_data
0,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is aggressively pursuing its growth strategy by increasingly focusing on technologically more demanding HDI printed circuit boards PCBs .,prediction,llama-3.1-8b-instant,"id 084eec7989f0cfbbb94860b54ffffe5c0ea064f896a8dc4e4e69dfa1f4fb6eb0\nblob NaN\nmeta {'sentiment': 'positive', 'source_id': '2c106b768ca3455523070e94cfcdbc9e145ca808ae79f54e5d2f7c56e21ab280', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0, '_split_overlap': []}\nscore ..."
1,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is aggressively pursuing its growth strategy by increasingly focusing on technologically more demanding HDI printed circuit boards PCBs .,Non-prediction,llama-3.3-70b-versatile,"id 084eec7989f0cfbbb94860b54ffffe5c0ea064f896a8dc4e4e69dfa1f4fb6eb0\nblob NaN\nmeta {'sentiment': 'positive', 'source_id': '2c106b768ca3455523070e94cfcdbc9e145ca808ae79f54e5d2f7c56e21ab280', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0, '_split_overlap': []}\nscore ..."
2,"Net sales surged by 18.5 % to EUR167 .8 m. Teleste said that EUR20 .4 m , or 12.2 % , of the sales came from the acquisitions made in 2009 .",prediction,llama-3.1-8b-instant,id 53f4afca60c6d97e69fe7bf80b319601dfaed593c...
3,"Net sales surged by 18.5 % to EUR167 .8 m. Teleste said that EUR20 .4 m , or 12.2 % , of the sales came from the acquisitions made in 2009 .",Non-prediction,llama-3.3-70b-versatile,id 53f4afca60c6d97e69fe7bf80b319601dfaed593c...
4,"According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .",prediction,llama-3.1-8b-instant,id c8250eb1bfc021b035d71ba172e262a21ab427b75...
5,"According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .",Prediction,llama-3.3-70b-versatile,id c8250eb1bfc021b035d71ba172e262a21ab427b75...
6,"The international electronic industry company Elcoteq has laid off tens of employees from its Tallinn facility ; contrary to earlier layoffs the company contracted the ranks of its office workers , the daily Postimees reported .",prediction,llama-3.1-8b-instant,id bc1d4924ecbc2356699cdec2c4662ea3f2125a2dc...
7,"The international electronic industry company Elcoteq has laid off tens of employees from its Tallinn facility ; contrary to earlier layoffs the company contracted the ranks of its office workers , the daily Postimees reported .",Non-prediction,llama-3.3-70b-versatile,id bc1d4924ecbc2356699cdec2c4662ea3f2125a2dc...
8,"Finnish Talentum reports its operating profit increased to EUR 20.5 mn in 2005 from EUR 9.3 mn in 2004 , and net sales totaled EUR 103.3 mn , up from EUR 96.4 mn .",prediction,llama-3.1-8b-instant,"id cda0ab041b78439da23a9559adf5a33bc578eb7080be3bf050270d043d491cce\nblob NaN\nmeta {'sentiment': 'positive', 'source_id': 'c4fff1f5fb136e3170ef2af33b26fe54a73fcf7f1f8feb8458dcf57c6fd3d7f8', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0, '_split_overlap': []}\nscore ..."
9,"Finnish Talentum reports its operating profit increased to EUR 20.5 mn in 2005 from EUR 9.3 mn in 2004 , and net sales totaled EUR 103.3 mn , up from EUR 96.4 mn .",Non-prediction,llama-3.3-70b-versatile,"id cda0ab041b78439da23a9559adf5a33bc578eb7080be3bf050270d043d491cce\nblob NaN\nmeta {'sentiment': 'positive', 'source_id': 'c4fff1f5fb136e3170ef2af33b26fe54a73fcf7f1f8feb8458dcf57c6fd3d7f8', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0, '_split_overlap': []}\nscore ..."


In [ ]:
path = os.path.join(notebook_dir, "../data/rag/retrieved/")
prefix = "text_label_name_meta_data"
save_file_type = "csv"
DataProcessing.save_to_file(updated_df, path, prefix, save_file_type)